# Import Library

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import re
from os import getcwd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cwd = getcwd() + "/drive/Shareddrives/CAPSTONE_ML/"
print(cwd)

/content/drive/Shareddrives/CAPSTONE_ML/


In [ ]:
df_all = pd.read_excel(cwd + "df_all_new.xlsx")
df_all.head()

,Page_URL,Shop_name,Rating,Reviews,Kategori,Address,Phone_Number,Opening_Hours,Range_Harga,Latitude,Longitude,Eco_Friendly,Categorize_Weather
0,https://www.google.com/maps/place/Wedang+Uwuh+...,Wedang Uwuh Solo - Pak Aden,4.6,5,Wedang Uwuh,"Mutihan rt 04 rw 1, Surakarta, Sondakan, Kec. ...",082324006000,"{'Senin': ['08.00–20.00'], 'Selasa': ['08.00–2...",Rp 1–25 rb,-7.566731,110.716919,0,Dingin
1,https://www.google.com/maps/place/Rajanya+JAHE...,Rajanya JAHE REMPAH wedhang uwuh Sumber,4.6,32,Wedang Uwuh,"Jl. Letjen Suprapto No.76c, Sumber, Kec. Banja...",085641727445,"{'Minggu': ['Buka 24 jam'], 'Senin': ['Buka 24...",Rp 1–25 rb,-7.566731,110.716919,0,Dingin
2,https://www.google.com/maps/place/Wedang+uwuh+...,Wedang uwuh jelantik solo,5.0,1,Wedang Uwuh,"Jl. Sungai Musi No.1, Sangkrah, Kec. Ps. Kliwo...",081229070267,"{'Senin': ['Buka 24 jam'], 'Selasa': ['Buka 24...",Rp 1–25 rb,-7.576765,110.768229,0,Dingin
3,https://www.google.com/maps/place/Wedang+uwuh+...,Wedang uwuh mbok tuo,4.9,14,Wedang Uwuh,"Jl. Kutai Tim. 5B, RT.05/RW.08, Sumber, Kec. B...",081578614885,"{'Senin': ['07.00–19.00'], 'Selasa': ['07.00–1...",Rp 1–25 rb,-7.547894,110.730811,0,Dingin
4,https://www.google.com/maps/place/wedang+uwuh+...,wedang uwuh sya,4.7,3,Wedang Uwuh,"Jl. Reksoniten No.20, Gajahan, Kec. Ps. Kliwon...",089608149211,"{'Minggu': ['Buka 24 jam'], 'Senin': ['Buka 24...",Rp 1–25 rb,-7.547894,110.730811,0,Dingin


Fungsi untuk menghitung jarak menggunakan formula haversine

In [ ]:
def calculate_haversine(lat1, lon1, lat2, lon2):
    earth_radius = 6371  # Radius bumi dalam kilometer
    delta_lat = np.radians(lat2 - lat1)
    delta_lon = np.radians(lon2 - lon1)
    a = np.sin(delta_lat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(delta_lon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return earth_radius * c

Menghitung jarak tiap lokasi berdasarkan input latitude dan longitude pengguna

In [ ]:
# Lokasi pengguna
current_lat = -7.554726966145416
current_lon = 110.85157469256261

# Perhitungan jarak untuk setiap lokasi dalam dataset
df_all['computed_distance'] = df_all.apply(
    lambda row: calculate_haversine(current_lat, current_lon, row['Latitude'], row['Longitude']), axis=1
)

In [ ]:
df_all['Categorize_Weather'].value_counts()

C = df_all['Rating'].mean()
M = df_all['Reviews'].quantile(0.5)
df_all['weighted_rating'] = ((df_all['Reviews'] / (df_all['Reviews'] + M)) * df_all['Rating']) + ((M / (df_all['Reviews'] + M)) * C)

In [ ]:
df_all.dtypes

,0
Page_URL,object
Shop_name,object
Rating,float64
Reviews,int64
Kategori,object
Address,object
Phone_Number,object
Opening_Hours,object
Range_Harga,object
Latitude,float64


In [ ]:
selected_df = df_all[['Categorize_Weather', 'weighted_rating', 'Kategori', 'Rating']]

# Split train dan test
train, val = train_test_split(selected_df, test_size=0.2, random_state=42)

print(len(train), 'train examples')
print(len(val), 'validation examples')

485 train examples
122 validation examples


In [ ]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()

    labels = dataframe.pop('Rating')

    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))

    if shuffle:
      ds = ds.shuffle(4)

    ds = ds.batch(batch_size)

    return ds

In [ ]:
feature_columns = []

numeric_columns = ['weighted_rating']

for header in numeric_columns:
  feature_columns.append(tf.feature_column.numeric_column(header))

weather_class = ['Dingin', 'Dingin/Panas', 'Panas']
thal_weather = tf.feature_column.categorical_column_with_vocabulary_list(key="weather", vocabulary_list=weather_class)
one_hot_weather = tf.feature_column.indicator_column(thal_weather)

feature_columns.append(one_hot_weather)

category_class = df_all['Kategori'].unique()
thal_category = tf.feature_column.categorical_column_with_vocabulary_list(key="category", vocabulary_list=category_class)
one_hot_category = tf.feature_column.indicator_column(thal_category)

feature_columns.append(one_hot_category)

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
#feature_layer = tf.compat.v1.keras.layers.DenseFeatures(feature_columns)

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'DenseFeatures'

In [ ]:
train_ds = df_to_dataset(train)
val_ds = df_to_dataset(val, shuffle=False)
# test_ds = df_to_dataset(test, shuffle=False)

In [ ]:
model = tf.models.Sequential([
    feature_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mape'])

AttributeError: module 'tensorflow' has no attribute 'models'

In [ ]:
model.fit(train_ds,
          validation_data=val_ds,
          epochs=100)

Epoch 1/100


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("sequential_1/Cast:0", shape=(None,), dtype=float32). Expected shape (None, 21), but input has incompatible shape (None,)[0m

Arguments received by Sequential.call():
  • inputs={'Categorize_Weather': 'tf.Tensor(shape=(None,), dtype=string)', 'weighted_rating': 'tf.Tensor(shape=(None,), dtype=float32)', 'Kategori': 'tf.Tensor(shape=(None,), dtype=string)'}
  • training=True
  • mask={'Categorize_Weather': 'None', 'weighted_rating': 'None', 'Kategori': 'None'}

# Menggunakan input np hstack

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df_all['Kategori_encoded'] = label_encoder.fit_transform(df_all['Kategori'])
category = tf.keras.utils.to_categorical(df_all['Kategori_encoded'])

df_all['Categorize_Weather_encoded'] = label_encoder.fit_transform(df_all['Categorize_Weather'])
weather = tf.keras.utils.to_categorical(df_all['Categorize_Weather_encoded'])

In [ ]:
C = df_all['Rating'].mean()
M = df_all['Reviews'].quantile(0.5)
df_all['weighted_rating'] = ((df_all['Reviews'] / (df_all['Reviews'] + M)) * df_all['Rating']) + ((M / (df_all['Reviews'] + M)) * C)

# df_all['weighted_rating'] = (df_all['Rating'] * np.log(df_all['Reviews'] + 1))

In [ ]:
feature_input = np.hstack([category, weather, np.expand_dims(df_all['weighted_rating'].values, axis=1)])

In [ ]:
feature_tensor = tf.convert_to_tensor(feature_input, dtype=tf.float32)

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(feature_tensor.shape[1],)),
    # tf.keras.layers.Dense(512, activation='relu'),
    # tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

target = df_all['Rating'].values

model.fit(feature_tensor, target, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 11.5888 - mae: 3.3264 - val_loss: 9.1921 - val_mae: 2.9891
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.4348 - mae: 2.6981 - val_loss: 5.9749 - val_mae: 2.4237
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.2356 - mae: 2.0220 - val_loss: 3.8016 - val_mae: 1.8992
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2.2913 - mae: 1.3919 - val_loss: 2.5601 - val_mae: 1.4481
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.4300 - mae: 0.8951 - val_loss: 2.0538 - val_mae: 1.1492
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3022 - mae: 0.6168 - val_loss: 1.9322 - val_mae: 1.0399
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.3919 - mae: 0.5624 - val_loss: 1.9439 - val_mae: 1.0530
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.5071 - mae: 0.6401 - val_loss: 2.0019 - val_mae: 1.1065
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.5

In [ ]:
model.save(cwd + '/copy_model_ml.keras')
model.save(cwd + '/copy_model_ml.h5')

In [ ]:
test_loss, test_mae = model.evaluate(feature_tensor, target, verbose=0)
print(f"Mean Absolute Error pada data uji: {test_mae}")

Mean Absolute Error pada data uji: 0.6659553647041321


In [ ]:
# Prediksi menggunakan model yang sudah dilatih
predictions = model.predict(feature_tensor)  # Hasil prediksi berupa array

# Add prediksi ke df_all sebagai kolom baru
df_all['toprecommendation'] = predictions

19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [ ]:
top_recommendations = df_all.sort_values(by='toprecommendation', ascending=False).head(607)
print(top_recommendations[['Shop_name', 'Kategori', 'Categorize_Weather', 'weighted_rating', 'Rating', 'Reviews', 'toprecommendation']])

                                         Shop_name          Kategori  \
266                   Warung Selat & Sop "LEZATIE"             Selat   
378                          Nasi Liwet Mbak Yanti        Sego Liwet   
369                         Nasi liwet mbak endang        Sego Liwet   
485                                 Sate Pak Pledi       Sate Buntel   
363                          Nasi Liwet Mbah Kisut        Sego Liwet   
..                                             ...               ...   
596              Warung Es Dawet Selasih Asli Solo  Es Dawet Telasih   
572  Es Kapal Depan Museum Keris Stadion Sriwedari          Es Kapal   
569                            Es Kapal Batik Solo          Es Kapal   
571                          Es Kapal Pinggir Kali          Es Kapal   
373         Nasi Liwet dan Ayam Geprek Mbak Laksmi        Sego Liwet   

    Categorize_Weather  weighted_rating  Rating  Reviews  toprecommendation  
266             Dingin         4.915769     5.0      173 

In [ ]:
top_nasi_liwet = top_recommendations[top_recommendations['Kategori'] == 'Sego Liwet'].sort_values(by='toprecommendation', ascending=False).head(10)
print(top_nasi_liwet[['Shop_name', 'Kategori', 'Categorize_Weather', 'weighted_rating', 'Rating', 'Reviews', 'toprecommendation']])

# top_recommendations[top_recommendations['Kategori'] == 'Sego Liwet']

                              Shop_name    Kategori Categorize_Weather  \
378               Nasi Liwet Mbak Yanti  Sego Liwet             Dingin   
369              Nasi liwet mbak endang  Sego Liwet             Dingin   
363               Nasi Liwet Mbah Kisut  Sego Liwet             Dingin   
345                   Nasi Liwet Sukini  Sego Liwet             Dingin   
310  Nasi Liwet & Cabuk Rambak Bu Parmi  Sego Liwet             Dingin   
341  Nasi Liwet & Bubur Lemu Mbah Narti  Sego Liwet             Dingin   
326        Nasi Liwet Mbak Marni Gentan  Sego Liwet             Dingin   
331     Nasi Liwet MBAK GIYEM Solo Baru  Sego Liwet             Dingin   
325                 Nasi Liwet Bu Darmi  Sego Liwet             Dingin   
346      nasi liwet solo pagi mbak atin  Sego Liwet             Dingin   

     weighted_rating  Rating  Reviews  toprecommendation  
378         4.655013     4.9       35           4.741651  
369         4.641650     4.9       32           4.731614  
363     

In [ ]:
top_sate_kere = top_recommendations[top_recommendations['Kategori'] == 'Sate Kere'].sort_values(by='toprecommendation', ascending=False).head(10)
print(top_sate_kere[['Shop_name', 'Kategori', 'Categorize_Weather', 'weighted_rating', 'Rating', 'Reviews', 'toprecommendation']])

                                             Shop_name   Kategori  \
384                                  Sate Kere Barokah  Sate Kere   
410                             Sate Kere Sastro Oleng  Sate Kere   
394                                  Sate Kere Bu Tini  Sate Kere   
382                     SATE KERE "MAK NARTI " Widuran  Sate Kere   
440                                 Sate Kere Mbah Yem  Sate Kere   
402                               SATE KERE BU MUJIYEM  Sate Kere   
400  Wedangan Kresna & Sate Kere Solo "Lombok Rawit...  Sate Kere   
437                     SEKECOW SATE KERE DAN WEDANGAN  Sate Kere   
391                Sate kere Mbak Tug ( Sate Presiden)  Sate Kere   
401                             Sate Kere Pasar Burung  Sate Kere   

    Categorize_Weather  weighted_rating  Rating  Reviews  toprecommendation  
384       Dingin/Panas         4.610016     4.9       26           4.069329  
410       Dingin/Panas         4.597676     4.9       24           4.059618  
394   

In [ ]:
top_wedang_uwuh = top_recommendations[top_recommendations['Kategori'] == 'Wedang Uwuh'].sort_values(by='toprecommendation', ascending=False).head(10)
print(top_wedang_uwuh[['Shop_name', 'Kategori', 'Categorize_Weather', 'weighted_rating', 'Rating', 'Reviews', 'toprecommendation']])


                                   Shop_name     Kategori Categorize_Weather  \
17                            Wedang Uwuh JD  Wedang Uwuh             Dingin   
8   Wedang Jahe Merah Keraton Cabang Coyudan  Wedang Uwuh             Dingin   
45                             Kedai Bedjono  Wedang Uwuh             Dingin   
21            OW Omah Wedangan (lokasi baru)  Wedang Uwuh             Dingin   
25               Wedangan Echo(honggowongso)  Wedang Uwuh             Dingin   
50                         Cafe Bukuku Lawas  Wedang Uwuh             Dingin   
59                       Wedangan Mas Bonnie  Wedang Uwuh             Dingin   
47                  Angkringan Babe Haryanto  Wedang Uwuh             Dingin   
23                          Wedangan Dhe Man  Wedang Uwuh             Dingin   
3                       Wedang uwuh mbok tuo  Wedang Uwuh             Dingin   

    weighted_rating  Rating  Reviews  toprecommendation  
17         4.705192     5.0       33           4.406546  
8  